In [98]:
import cfscrape 
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [99]:
def get_data(url, currency='no'):
    today = pd.to_datetime(datetime.now().date())
    
    scraper = cfscrape.create_scraper() 
    scraped_data = scraper.get(url) 
    
    soup = BeautifulSoup(scraped_data.text, 'html.parser')

    data = soup.find_all('td', class_='datatable_cell__LJp3C', dir="ltr")
    time = soup.find_all('time')
    data_temp_list = [i.text for i in data]

    data_list = []

    for i in data_temp_list:
        i = i.replace('.', '')
        i = i.replace(',', '.')
        data_list.append(i)

    data = []

    for i in data_list:
        try:
            i = pd.to_numeric(i)
            data.append(i)
        except:
            pass

    if currency == 'yes':
        data = data[:23]

    else:
        pass

    time_temp_list = [i.text for i in time]
    time_temp_list = time_temp_list[1:]

    result = pd.DataFrame(data={'date': time_temp_list, 'price': data})
    result['date'] = pd.to_datetime(result['date'], dayfirst=True)

    result = result[result['date'] != today]

    result.drop_duplicates(inplace=True)
    result.reset_index(inplace=True, drop=True)

    return result

In [ ]:
def investin_com():
    
    def get_data(url, currency='no'):
        today = pd.to_datetime(datetime.now().date())
        
        scraper = cfscrape.create_scraper() 
        scraped_data = scraper.get(url) 
        
        soup = BeautifulSoup(scraped_data.text, 'html.parser')

        data = soup.find_all('td', class_='datatable_cell__LJp3C', dir="ltr")
        time = soup.find_all('time')
        data_temp_list = [i.text for i in data]

        data_list = []

        for i in data_temp_list:
            i = i.replace('.', '')
            i = i.replace(',', '.')
            data_list.append(i)

        data = []

        for i in data_list:
            try:
                i = pd.to_numeric(i)
                data.append(i)
            except:
                pass

        if currency == 'yes':
            data = data[:23]

        else:
            pass

        time_temp_list = [i.text for i in time]
        time_temp_list = time_temp_list[1:]

        result = pd.DataFrame(data={'date': time_temp_list, 'price': data})
        result['date'] = pd.to_datetime(result['date'], dayfirst=True)

        result = result[result['date'] != today]

        result.drop_duplicates(inplace=True)
        result.reset_index(inplace=True, drop=True)

        return result
    
    gdaxi = get_data('https://ru.investing.com/indices/germany-30-historical-data')
    historical_data = pd.read_excel('./data/investing_com/GDAXI.xlsx', index_col=0)
    
    imoex = get_data('https://ru.investing.com/indices/mcx-historical-data')
    
    spx = get_data('https://ru.investing.com/indices/us-spx-500-historical-data')
    
    ssek = get_data('https://ru.investing.com/indices/shanghai-composite-historical-data')
    
    urals = get_data('https://ru.investing.com/commodities/crude-oil-urals-spot-futures-historical-data')
    
    currency = get_data('https://ru.investing.com/currencies/eur-usd-historical-data', currency='yes')
    
    pass

In [101]:
gdaxi = get_data('https://ru.investing.com/indices/germany-30-historical-data')
imoex = get_data('https://ru.investing.com/indices/mcx-historical-data')
spx = get_data('https://ru.investing.com/indices/us-spx-500-historical-data')
ssek = get_data('https://ru.investing.com/indices/shanghai-composite-historical-data')
urals = get_data('https://ru.investing.com/commodities/crude-oil-urals-spot-futures-historical-data')
#currency = get_data('https://ru.investing.com/currencies/eur-usd-historical-data', currency='yes')

In [102]:
gdaxi

,date,price
0,2023-11-01,14923.27
1,2023-10-31,14810.34
2,2023-10-30,14716.54
3,2023-10-27,14687.41
4,2023-10-26,14731.05
5,2023-10-25,14892.18
6,2023-10-24,14879.94
7,2023-10-23,14800.72
8,2023-10-20,14798.47
9,2023-10-19,15045.23


In [95]:
df = pd.read_excel('./data/investing_com/GDAXI.xlsx', index_col=0)
df

,date,price
1,2023-10-31,14810.34
2,2023-10-30,14716.54
3,2023-10-27,14687.41
4,2023-10-26,14731.05
5,2023-10-25,14892.18
...,...,...
4024,2008-01-08,7849.99
4025,2008-01-07,7817.17
4026,2008-01-04,7808.69
4027,2008-01-03,7908.41


In [96]:
test = pd.concat([gdaxi, df], axis=0).reset_index(drop=True).drop_duplicates('date')

In [97]:
with pd.ExcelWriter(
    "./data/investing_com/GDAXI_test.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD") as writer:
    test.to_excel(writer, sheet_name='GDAXI')